In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from util import get_data

import TheoreticallyOptimalStrategy as TOS
from marketsimcode import compute_portvals, compute_stats
import indicators
from strategy_evaluation.ManualStrategy import ManualStrategy
from strategy_evaluation.StrategyLearner import StrategyLearner

#STRATEGY LEARNER

In [17]:
symbol = "JPM"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.005

strategylearner1 = StrategyLearner(impact=impact, commission=commission)
strategylearner1.add_evidence(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
trades1 = strategylearner1.testPolicy(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
portvals1 = compute_portvals(trades1, start_val=sv, commission=commission, impact=impact)
# portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)
print(compute_stats(portvals1))

strategylearner2 = StrategyLearner(impact=impact, commission=commission)
strategylearner2.add_evidence(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
trades2 = strategylearner2.testPolicy(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
portvals2 = compute_portvals(trades2, start_val=sv, commission=commission, impact=impact)
# portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)
print(compute_stats(portvals2))

strategylearner3 = StrategyLearner(impact=impact, commission=commission)
strategylearner3.add_evidence(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
trades3 = strategylearner3.testPolicy(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
portvals3 = compute_portvals(trades3, start_val=sv, commission=commission, impact=impact)
# portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)
print(compute_stats(portvals3))

q1-table coverage:  0.38271604938271603
            JPM
2008-01-02    2
2008-01-03    2
2008-01-04    0
2008-01-07    1
2008-01-08    0
...         ...
2009-12-24    2
2009-12-28    0
2009-12-29    2
2009-12-30    2
2009-12-31    2

[505 rows x 1 columns]
(0.0029880448146684113, 0.00012955339192832803, 0.01576015596633924, 0.13049340017166958)
q1-table coverage:  0.4074074074074074
            JPM
2008-01-02    1
2008-01-03    1
2008-01-04    1
2008-01-07    1
2008-01-08    1
...         ...
2009-12-24    0
2009-12-28    0
2009-12-29    1
2009-12-30    1
2009-12-31    1

[505 rows x 1 columns]
(0.11075305342708774, 0.00030979282652763527, 0.014302528886114425, 0.3438418968297152)
q1-table coverage:  0.41975308641975306
            JPM
2008-01-02    1
2008-01-03    1
2008-01-04    1
2008-01-07    1
2008-01-08    1
...         ...
2009-12-24    2
2009-12-28    0
2009-12-29    2
2009-12-30    2
2009-12-31    2

[505 rows x 1 columns]
(0.09989258269479695, 0.00030085497879437463, 0.0150118

#MANUAL STRATEGY

In [4]:
# Manual Strategy

symbol = "JPM"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.005

# sd = out_sample_sd
# ed = out_sample_ed

sd = in_sample_sd
ed = in_sample_ed

indicator = indicators.Indicators(symbol, pd.date_range(sd, ed))
bbp = indicator.get_bbp(window = 20)
rsi = indicator.get_rsi(window = 14)
macd = indicator.get_macd().loc[sd:] # My version of indicators.py is also returning values 9 market days prior to sd
ppo = indicator.get_ppo()

# Calculate MACD crossover
macd_prev = macd.shift(1)
macd_prev.iloc[0] = macd_prev.iloc[1]
conditions = [
    (macd_prev <= 0) & (macd > 0),
    (macd_prev >= 0) & (macd < 0)
]
values = [1, -1]
# Documentation: https://numpy.org/doc/2.1/reference/generated/numpy.select.html
macd_crossover = pd.DataFrame({'result': np.select(conditions, values, default=0)}, index=macd.index, columns=["result"])

# Generate signals
# signals_cond = [
#     ((bbp <= 0) & (rsi <= 30)) | ((bbp <= 0) & (macd_crossover["result"] == 1)) | ((rsi <= 30) & (macd_crossover["result"] == 1)),
#     ((bbp >= 100) & (rsi >= 70)) | ((bbp >= 100) & (macd_crossover["result"] == 1)) | ((rsi >= 70) & (macd_crossover["result"] == 1))
# ]

signals_cond = [
            ((bbp <= 0) & (rsi <= 30)) | ((bbp <= 0) & (ppo > 1)) | ((rsi <= 30) & (ppo > 1)),
            ((bbp >= 100) & (rsi >= 70)) | ((bbp >= 100) & (ppo < -1)) | ((rsi >= 70) & (ppo < -1))
        ]
signal_values = [1, -1]
signals = pd.DataFrame({'result': np.select(signals_cond, signal_values, default=0)}, index=bbp.index)

# Based on indicators on T day, we will place orders on T+1 day
signals = signals.shift(1)
signals = signals.fillna(0)
signals.drop(signals.index[-1])

# Generate Manual Strategy trades
net_position = 0
manual_trades = pd.DataFrame(0, index=bbp.index, columns=[symbol])

for date in signals.index:
    if signals.loc[date].iloc[0] == 1:
        manual_trades.loc[date] = 1000 - net_position
    elif signals.loc[date].iloc[0] == -1:
        manual_trades.loc[date] = -1000 - net_position

    net_position += manual_trades.loc[pd.to_datetime(date), symbol]

# portvals = compute_portvals(manual_trades, start_val=sv, commission=commission, impact=impact)
portvals = compute_portvals(manual_trades, start_val=sv, commission=commission, impact=impact)

print(compute_stats(portvals))

# manual_trades
ppo

(0.49339250000000034, 0.0009023636801227652, 0.01473705849458871, 0.9720107539581988)


2008-01-02   -0.804646
2008-01-03   -1.141377
2008-01-04   -1.655662
2008-01-07   -1.893454
2008-01-08   -2.532519
                ...   
2009-12-24   -0.792660
2009-12-28   -0.676097
2009-12-29   -0.649488
2009-12-30   -0.607508
2009-12-31   -0.526184
Name: JPM, Length: 505, dtype: float64

In [36]:
compute_stats(portvals)

(-0.570665, -0.0016337761780937103, 0.05524187128897186, -0.46948794792624565)

In [18]:
symbol = "SINE_FAST_NOISE"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.005

# sd = out_sample_sd
# ed = out_sample_ed

sd = in_sample_sd
ed = in_sample_ed

indicator = indicators.Indicators(symbol, pd.date_range(sd, ed))
bbp = indicator.get_bbp(window = 20)
rsi = indicator.get_rsi(window = 14)
macd = indicator.get_macd().loc[sd:] # My version of indicators.py is also returning values 9 market days prior to sd
ppo = indicator.get_ppo()
price = get_data([symbol], pd.date_range(sd, ed))[symbol]

# price
# bbp
# rsi
# macd
# ppo

strategylearner = StrategyLearner(impact=impact, commission=commission)
list = pd.DataFrame(0, index=price.index, columns=[symbol])
for date in bbp.index:
    list.loc[date] = strategylearner.discretize(bbp.loc[date], rsi.loc[date], ppo.loc[date])

price

2008-01-02     96.211521
2008-01-03    105.146639
2008-01-04    107.390173
2008-01-07    100.964659
2008-01-08    106.917580
                 ...    
2009-12-24    105.904712
2009-12-28     95.060619
2009-12-29    112.276783
2009-12-30    105.471881
2009-12-31    102.257554
Name: SINE_FAST_NOISE, Length: 505, dtype: float64

In [15]:
manual_trades.loc[manual_trades[symbol] <= -1000].index

DatetimeIndex(['2008-07-18', '2009-02-10', '2009-05-11', '2009-08-06'], dtype='datetime64[ns]', freq=None)